In [1]:
import numpy as np
import os
import time
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


import keras
import keras.applications as kapp
from keras.datasets import cifar10

(x_train, y_train_cats), (x_test, y_test_cats) = cifar10.load_data()
batch_size = 8
x_train = x_train[:batch_size]
x_train = np.repeat(np.repeat(x_train, 7, axis=1), 7, axis=2)
model = kapp.VGG19()
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Running initial batch (compiling tile program)")
y = model.predict(x=x_train, batch_size=batch_size)

# Now start the clock and run 10 batches
print("Timing inference...")
start = time.time()
for i in range(10):
    y = model.predict(x=x_train, batch_size=batch_size)
print("Ran in {} seconds".format(time.time() - start))

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


Running initial batch (compiling tile program)
Timing inference...
Ran in 2.8074259757995605 seconds


In [4]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.getcwd()

'/Users/leonard/Google Drive Active(leonard.ng.wt@gmail.com)/rsna'

In [5]:
import matplotlib.pyplot as plt
import glob, pylab, pandas as pd
import numpy as np
%matplotlib inline

from keras.preprocessing.image import ImageDataGenerator

In [6]:
df_detailed = pd.read_csv('./stage_2_detailed_class_info.csv')

In [7]:
df_detailed.head()

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity


In [8]:
df_detailed['patientId'] =   df_detailed['patientId'].astype(str) + '.png' 

In [9]:
df_detailed.head()

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6.png,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd.png,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb.png,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5.png,Normal
4,00436515-870c-4b36-a041-de91049b9ab4.png,Lung Opacity


In [10]:
image_gen = ImageDataGenerator(rotation_range = 30,
                              width_shift_range= 0.1, 
                              height_shift_range= 0.1,
                              rescale= 1/255,
                              shear_range=0.2,
                              zoom_range= 0.2,
                              horizontal_flip= True,
                              fill_mode='nearest',
                               validation_split =0.2
                              )

In [11]:
train_image_gen = image_gen.flow_from_dataframe(dataframe = df_detailed,
                             directory= './train',
                             x_col = 'patientId',
                             y_col = 'class',
                              target_size=(224,224),
                              color_mode='rgb',
                              classes= None,
                              class_mode='categorical',
                              batch_size=32,
                              shuffle=True,
                              subset = 'training'
                             )

Found 24015 validated image filenames belonging to 3 classes.


/Users/leonard/opt/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 209 invalid image filename(s) in x_col="patientId". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [12]:
test_image_gen = image_gen.flow_from_dataframe(dataframe = df_detailed,
                             directory= './train',
                             x_col = 'patientId',
                             y_col = 'class',
                              target_size=(224,224),
                              color_mode='rgb',
                              classes= None,
                              class_mode='categorical',
                              batch_size=32,
                              shuffle=True,
                              subset = 'validation'
                             )

Found 6003 validated image filenames belonging to 3 classes.


In [13]:
!pip install InceptionResNetV2

ERROR: Could not find a version that satisfies the requirement InceptionResNetV2 (from versions: none)
ERROR: No matching distribution found for InceptionResNetV2


In [14]:
from tensorflow import keras
from keras.applications import InceptionResNetV2
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Conv3D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.models import Model

In [15]:
orig_net = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3)) 


AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [22]:
filters = GlobalAveragePooling2D()(orig_net.output) 


In [23]:
classifiers = Dense(3, activation='softmax', bias_initializer='ones')(filters)  

In [24]:
model = Model(inputs=orig_net.inputs, outputs=classifiers)  


In [25]:
model.compile(loss='categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [28]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("dense121_3.h5", monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto',
                             period=1)

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=20, 
                      verbose=1, 
                      mode='auto')

results = model.fit_generator(train_image_gen, epochs =150, 
                              steps_per_epoch=400,
                             validation_data=test_image_gen,  
                              callbacks=[checkpoint,early])

Epoch 1/150
400/400 [==============================] - 4644s 12s/step - loss: nan - acc: 0.2920 - val_loss: nan - val_acc: 0.4233

Epoch 00001: val_acc improved from -inf to 0.42329, saving model to dense121_3.h5
Epoch 2/150
  5/400 [..............................] - ETA: 1:04:11 - loss: nan - acc: 0.3187

KeyboardInterrupt: 